In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import numpy as np
import cv2
import random
import pathmagic  # noqa
from panorama.config_gen import PanoramaConfig
from panorama.net.net import PanoramaNet

## Setting up Panorama

In [4]:
class args:
    config_path = '../../trained_models/faces_config.json'
    k = 5
    model_path = \
        '../../trained_models/panorama_faces_original_loss_weights.h5'
    model_qualification_path = \
        '../../trained_models/panorama_faces_original_loss_weights.csv'
    target = 0.9
    margin = 44
    album_path = '../../trained_models/ytf_album.pkl'
    

### Loading configs

In [5]:
config_gen = PanoramaConfig(args.config_path,
                                '',
                                '',
                                '',
                                '',
                                '',
                                args.model_path,
                                is_force=False
                                )

config = config_gen.get_config()
random.seed(config['random_seed'])

### Initializing PanoramaNet

In [6]:
panoramaNet = PanoramaNet(config)
panoramaNet.load_weights(args.model_path)
panoramaNet.set_cas_param(args.model_qualification_path, auto=True)

Loading weights from ../../trained_models/panorama_faces_original_loss_weights.h5


## Verification example
Pass in a pair of frame and ask if there are any common identities depicted.

In [7]:
frame_pair = ('./frame_0000147691.jpg', './frame_0000158434.jpg')

In [8]:
result, is_RM, short_circuited_at = panoramaNet.verify(frame_pair)

In [9]:
print(result, short_circuited_at)

True 0


Result is ```True``` and the query was short-circuited at ```Block_0``` and reference model was not needed.

## Recognition example
Pass in a single frame and an album of known objects to ask for the labels. First load the album into k-nn models. Pass ```voc=None``` meaning setting the vocabulary to be whatever is contained by the album.

In [11]:
neigh, voc = panoramaNet.load_album(args.album_path, voc=None, k=args.k)

No voc given


### Running recognition
This is one frame from YoutubeFaces dataset depicting Osama Bin Laden. The album is also polled from the same dataset having a vocabulary size of 1588. This album contains frames from the same video as does our query.

In [12]:
filename = "./Osama_bin_Laden_5_5.73.jpg"

In [13]:
result = panoramaNet.recognize(filename, neigh, k=args.k)

The scores here are the distances between identities. The lower the more confident.

In [14]:
result

[('Osama_bin_Laden', 0.123167075),
 ('Patrick_McEnroe', 0.8321626),
 ('Chris_Cookson', 0.8444098),
 ('Jack_Osbourne', 0.87500244),
 ('Terrence_Trammell', 0.90793765)]